In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os, sys

path = os.path.abspath("/home/jb7410/scarl_home/scarl/scarl")
sys.path.append(path)

In [3]:
from Synthesizor import Synthesizor

/home/jb7410/anaconda3/envs/scarl2/lib/python3.9/site-packages/numpy/core/getlimits.py:549: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/jb7410/anaconda3/envs/scarl2/lib/python3.9/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/home/jb7410/anaconda3/envs/scarl2/lib/python3.9/site-packages/numpy/core/getlimits.py:549: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/jb7410/anaconda3/envs/scarl2/lib/python3.9/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  return self._float_to_str(self.smallest_subnormal)


In [4]:
class Employee(object):
    def __init__(self, _dict):
        self.__dict__.update(_dict)

In [5]:
run_params = {
        "NUM_LENGTH_RECIPE":18,
        "TOP_MODULE_NAME": 'aes128_table_ecb'
}
argsVals = {
        'file': "/home/jb7410/scarl_home/data/aes128_table_ecb_mod.v",
        'lib': "/home/jb7410/scarl_home/data/merge.lib",
        'dump':"/home/jb7410/scarl_home/dump/trial2",
        'params':run_params
}
logfile = "/home/jb7410/scarl_home/dump/trial2"

In [6]:
args = Employee(argsVals)
synthEnv = Synthesizor(args)
synthEnv.checkFilePathsAndCreateAig()
state = synthEnv.init_state()

In [7]:
synthEnv.state

{'node_type': tensor([0, 0, 0,  ..., 2, 2, 2]),
 'num_inverted_predecessors': tensor([0, 0, 0,  ..., 0, 1, 2]),
 'edge_index': tensor([[  911,   912,   913,  ..., 49542, 49543, 49543],
         [ 1692,  1695,  1698,  ...,   463, 49541, 49542]]),
 'x': tensor([[0, 0],
         [0, 0],
         [0, 0],
         ...,
         [2, 0],
         [2, 1],
         [2, 2]]),
 'edge_attr': tensor([[1],
         [1],
         [1],
         ...,
         [0],
         [0],
         [0]]),
 'nodes': 49544,
 'recipe_len': 0,
 'recipe_encoding': [],
 'aig_state': '/home/jb7410/scarl_home/dump/trial2/aig_states/orig_aig+0+step0.aig'}

In [49]:
import copy
state1 = copy.deepcopy(synthEnv.state)

In [ ]:
import pickle

with open('pyg_dict.pkl','wb') as f:
    pickle.dump(state1,f)

In [ ]:
import pickle,os

if os.path.exists('pyg_dict.pkl'):
    with open('pyg_dict.pkl','rb') as f:
        state1 = pickle.load(f)
    print(state1)

In [ ]:
random_action = synthEnv.action_space.sample()
obs, reward, terminated, truncated, info = synthEnv.step(random_action)

In [52]:
import torch_geometric

In [ ]:
state2 = copy.deepcopy(synthEnv.state)

In [100]:
from aig_feature_extractor import AIGStateEncoder
import torch_geometric

[autoreload of torch_geometric.nn.conv.sage_conv_SAGEConv_propagate failed: Traceback (most recent call last):
  File "/home/jb7410/anaconda3/envs/scarl2/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 276, in check
    superreload(m, reload, self.old_objects)
  File "/home/jb7410/anaconda3/envs/scarl2/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 475, in superreload
    module = reload(module)
  File "/home/jb7410/anaconda3/envs/scarl2/lib/python3.9/importlib/__init__.py", line 168, in reload
    raise ModuleNotFoundError(f"spec not found for the module {name!r}", name=name)
ModuleNotFoundError: spec not found for the module 'torch_geometric.nn.conv.sage_conv_SAGEConv_propagate'
]


In [92]:
state1['recipe_encoding'] = torch.tensor([1,5,6,7],dtype=torch.float32).reshape(1,-1,1)

In [93]:
state1

{'node_type': tensor([0, 0, 0,  ..., 2, 2, 2]),
 'num_inverted_predecessors': tensor([0, 0, 0,  ..., 0, 1, 2]),
 'edge_index': tensor([[  911,   912,   913,  ..., 49542, 49543, 49543],
         [ 1692,  1695,  1698,  ...,   463, 49541, 49542]]),
 'x': tensor([[0, 0],
         [0, 0],
         [0, 0],
         ...,
         [2, 0],
         [2, 1],
         [2, 2]]),
 'edge_attr': tensor([[1],
         [1],
         [1],
         ...,
         [0],
         [0],
         [0]]),
 'nodes': 49544,
 'recipe_len': 0,
 'recipe_encoding': tensor([[[1.],
          [5.],
          [6.],
          [7.]]]),
 'aig_state': '/home/jb7410/scarl_home/dump/trial2/aig_states/orig_aig+0+step0.aig'}

In [94]:
def dict_to_pyg(observation):
    """
    Convert an input observation to a PyTorch tensor that can be fed to a model.
    Includes sugar-coating to handle different observations (e.g. normalizing images).

    :param observation: the input observation
    :return: The observation as PyTorch tensor
        and whether the observation is vectorized or not
    """
    numNodes = observation['nodes']
    data = torch_geometric.data.Data.from_dict(observation)
    data.num_nodes = numNodes

    data = data.to('cuda')
    return data

In [95]:
data1 = dict_to_pyg(state1)

In [96]:
data1 = data1.sort()

## Checking the GNN model

In [101]:
model1 = AIGStateEncoder()

In [102]:
model1 = model1.cuda()

In [103]:
next(model1.parameters()).is_cuda

True

In [104]:
data1.recipe_encoding.shape

torch.Size([1, 4, 1])

In [106]:
model1(data1)

tensor([[ 0.0609,  0.1668, -0.1645, -0.1147, -0.0883, -0.0639,  0.0709,  0.0298,
         -0.0373,  0.1232,  0.1171,  0.0746,  0.0322, -0.1360, -0.0960,  0.0720,
          0.0053, -0.1463,  0.0989, -0.0021,  0.0646,  0.0773, -0.1101,  0.0850,
          0.0941,  0.1323,  0.0277, -0.0730,  0.0338, -0.0389,  0.0348, -0.0813,
          0.0445, -0.0205,  0.1174,  0.0803,  0.0733, -0.1589,  0.1584,  0.0921,
         -0.0843, -0.0011, -0.0325,  0.0596, -0.1355,  0.1230,  0.0111,  0.0353,
         -0.1464,  0.0696, -0.0538, -0.0355,  0.0221,  0.1010, -0.0080, -0.1231,
          0.0901,  0.0444,  0.0455,  0.0335,  0.1218,  0.0412,  0.1157,  0.0517]],
       device='cuda:0', grad_fn=<AddmmBackward0>)

In [ ]:
import torch
with torch.no_grad():
    torch.cuda.empty_cache()

In [9]:
from stable_baselines3.common.type_aliases import Schedule
def constant_fn(val: float) -> Schedule:
    """
    Create a function that returns a constant
    It is useful for learning rate schedule (to avoid code duplication)

    :param val: constant value
    :return: Constant schedule function.
    """

    def func(_):
        return val

    return func

In [8]:
from actor_critic_policy import ActorCriticPolicy

In [ ]:
import gc

#model.cpu()
del model1
gc.collect()
torch.cuda.empty_cache()

In [10]:
actrCriticPA = ActorCriticPolicy(synthEnv.action_space,lr_schedule=constant_fn(1))

In [12]:
list(actrCriticPA.parameters())
    

[autoreload of torch_geometric.nn.conv.sage_conv_SAGEConv_propagate failed: Traceback (most recent call last):
  File "/home/jb7410/anaconda3/envs/scarl2/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 276, in check
    superreload(m, reload, self.old_objects)
  File "/home/jb7410/anaconda3/envs/scarl2/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 475, in superreload
    module = reload(module)
  File "/home/jb7410/anaconda3/envs/scarl2/lib/python3.9/importlib/__init__.py", line 168, in reload
    raise ModuleNotFoundError(f"spec not found for the module {name!r}", name=name)
ModuleNotFoundError: spec not found for the module 'torch_geometric.nn.conv.sage_conv_SAGEConv_propagate'
]


[Parameter containing:
 tensor([[ 0.3935,  0.7384, -0.8965],
         [-0.5594, -0.0565, -0.7818],
         [-0.6530,  0.1869,  0.1189]], requires_grad=True),
 Parameter containing:
 tensor([0.1000], requires_grad=True),
 Parameter containing:
 tensor([[-0.3779,  0.1613,  0.1790,  0.1289],
         [-0.4955,  0.4735,  0.2063, -0.1231],
         [ 0.3606,  0.4662, -0.3950, -0.4507],
         [ 0.2379,  0.4436,  0.0160, -0.2445],
         [-0.4097,  0.4607, -0.3999,  0.3911],
         [ 0.4201, -0.0830,  0.2772,  0.1854],
         [-0.2123,  0.4269,  0.3201, -0.4880],
         [-0.4610,  0.1687,  0.2098,  0.0299],
         [ 0.2536, -0.1594, -0.1581, -0.4846],
         [-0.2470, -0.1574,  0.4735, -0.0780],
         [-0.0009,  0.0290, -0.1016,  0.4831],
         [-0.2112,  0.4197, -0.1983, -0.2922],
         [-0.1789, -0.1969, -0.4663, -0.3274],
         [-0.3721,  0.1899, -0.4382,  0.1777],
         [ 0.0448,  0.4714, -0.0092, -0.2296],
         [ 0.1280, -0.0758,  0.3437,  0.0942]], req

In [ ]:
actrCriticPA = actrCriticPA.to('cuda')

In [ ]:
actrCriticPA(data1)

In [ ]:
from PPO import PPO

In [ ]:
rl_model = PPO("AIGPolicy",synthEnv,verbose=1)

In [ ]:
rl_model.learn(total_timesteps=1000)

In [ ]:
synthEnv.init_state()

In [47]:
import torch
import torch.nn as nn

class SimpleLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(SimpleLSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        # Initialize hidden state with zeros
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        # Initialize cell state
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        
        # Forward propagate LSTM
        out, _ = self.lstm(x, (h0, c0))  # out: tensor of shape (batch_size, seq_length, hidden_size)
        
        # Decode the hidden state of the last time step
        out = self.fc(out[:, -1, :])
        print(out.shape)
        out = self.sigmoid(out)
        return out

In [48]:
# Example usage
input_size = 1  # Input size (dimension of input features)
hidden_size = 64  # Hidden size of the LSTM
num_layers = 1  # Number of layers in the LSTM
num_classes = 1  # Output size (dimension of the output)
model = SimpleLSTM(input_size, hidden_size, 2, num_classes)

# Example input data
#batch_size = 5
max_seq_length = 18
#input_data = torch.randn(batch_size, max_seq_length, input_size)
input_data = torch.randn(1,12, input_size)


# Forward pass
print(input_data.shape)
output = model(input_data)
print(output.shape)

torch.Size([1, 12, 1])
torch.Size([1, 1])
torch.Size([1, 1])


In [46]:
model(input_data)

<built-in method size of Tensor object at 0x7f16b3dcfd60>


tensor([[0.5020]], grad_fn=<SigmoidBackward0>)

In [33]:
sum(p.numel() for p in model.parameters())

50497

In [40]:
x = torch.tensor([i for i in range(11)])

In [41]:
model.lstm.weight_

tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10])

In [42]:
x.reshape(1,-1,1)

tensor([[[ 0],
         [ 1],
         [ 2],
         [ 3],
         [ 4],
         [ 5],
         [ 6],
         [ 7],
         [ 8],
         [ 9],
         [10]]])

In [37]:
x.size()

torch.Size([11])